## Once a support line is breached, there is a significant surge in volatility
    Comparing n-day returns after 50, 100 and 200 day moving averages are breached and checking if there is a significant increase in volatility

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_50 = pd.read_excel('Days_50dmaBreach.xlsx', parse_dates = ['DATE 50d MA is BREACHED'])

df_100 = pd.read_excel('Days_100dmaBreach.xlsx', parse_dates = ['DATE 100d MA is BREACHED'])

df_200 = pd.read_excel('Days_200dmaBreach.xlsx', parse_dates = ['Date'])

In [3]:
##Get Days when n-day moving averages are breached

In [4]:
breach50_days = list(df_50['DATE 50d MA is BREACHED'])

breach100_days = list(df_100['DATE 100d MA is BREACHED'])

breach200_days = list(df_200['Date'])

In [5]:
#Create masks

In [6]:
SP500ReturnsDF = pd.read_excel('SP500Returns.xlsx',parse_dates = ['DATE'] )

In [7]:
mask_50 =[True if SP500ReturnsDF.loc[i]['DATE'] in breach50_days else False for i in range(len(SP500ReturnsDF))]

mask_100 = [True if SP500ReturnsDF.loc[i]['DATE'] in breach100_days else False for i in range(len(SP500ReturnsDF))]

mask_200 = [True if SP500ReturnsDF.loc[i]['DATE'] in breach200_days else False for i in range(len(SP500ReturnsDF))]

In [8]:
#convenience function to get the nth day return after a breach
def giveNdayReturns(SP500ReturnsDF, n , mask):
    returns = []
    for i in range(len(mask)):
        if mask[i]:
            returns.append((SP500ReturnsDF.loc[i+n]['Close Price']/SP500ReturnsDF.loc[i]['Close Price'])-1)
    return returns

In [9]:
#n Day Returns in Dictionaries:
ret50_nday = {}
ret100_nday = {}
ret200_nday = {}

for i in range(1,41):
    ret50_nday['X+'+ str(i)] = giveNdayReturns(SP500ReturnsDF, i, mask_50)

for i in range(1,41):
    ret100_nday['X+'+ str(i)] = giveNdayReturns(SP500ReturnsDF, i, mask_100)

for i in range(1,41):
    ret200_nday['X+'+ str(i)] = giveNdayReturns(SP500ReturnsDF, i, mask_200)

In [10]:
# pd.DataFrame(ret50_nday).to_excel('dma50NdayReturns.xlsx')
# pd.DataFrame(ret100_nday).to_excel('dma100NdayReturns.xlsx')
# pd.DataFrame(ret200_nday).to_excel('dma200NdayReturns.xlsx')

In [11]:
# Using shift to calculate n-day returns of SP500 and storing them in a dictionary with key name 'X+n'
# we will be comparing this with the returns after a breach in the 50/100/200 day moving average
spNDayReturns = {}
for i in range(1,41):
    spNDayReturns['X+'+ str(i)] = (SP500ReturnsDF['Close Price']/ SP500ReturnsDF['Close Price'].shift(i))-1  

In [12]:
#pd.DataFrame(spNDayReturns).to_excel('sp500retsNday.xlsx')

In [13]:
spDF = pd.read_excel('sp500retsNday.xlsx')

In [14]:
spDF.head()

,Unnamed: 0,X+1,X+2,X+3,X+4,X+5,X+6,X+7,X+8,X+9,...,X+31,X+32,X+33,X+34,X+35,X+36,X+37,X+38,X+39,X+40
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.015421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.008383,0.023932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.019387,0.027933,0.043784,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,-0.004329,0.014975,0.023483,0.039265,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# removing nan values and storing it in a new dictionary
SPDFinal = {}
for i in range(1,41):
    SPDFinal['X+'+str(i)]= [i for i in list(spDF['X+'+str(i)]) if str(i) != 'nan']

In [16]:
#SPDFinal['X+2']

## Levene's test for homogeneity of variance

In [17]:

from scipy.stats import levene

In [18]:
levene50 = []
levene100 = []
levene200 = []
for i in range(1,41):
    levene50.append(levene(ret50_nday['X+'+str(i)], SPDFinal['X+' +str(i)]))
    levene100.append(levene(ret100_nday['X+'+str(i)], SPDFinal['X+' +str(i)]))
    levene200.append(levene(ret200_nday['X+'+str(i)], SPDFinal['X+' +str(i)]))

In [19]:
levene50pvals= [i[1] for i in levene50]
levene100pvals= [i[1] for i in levene100]
levene200pvals= [i[1] for i in levene200]

In [20]:
pvalue_dict = {'Holding period in days': range(1,41), 'Levenes test 50dayMA pvalue' :levene50pvals, 'Levenes test 100dayMA pvalue' :levene100pvals, 'Levenes test 200dayMA pvalue' :levene200pvals }

In [21]:
df_pvalues = pd.DataFrame(pvalue_dict, index = pvalue_dict['Holding period in days'])

In [22]:
df_pvalues.head()

,Holding period in days,Levenes test 50dayMA pvalue,Levenes test 100dayMA pvalue,Levenes test 200dayMA pvalue
1,1,0.805608,0.223170,0.416097
2,2,0.485225,0.122608,0.004445
3,3,0.162964,0.039003,0.003921
4,4,0.589118,0.104311,0.001943
5,5,0.440720,0.026825,0.011141


In [23]:
#df_pvals = pd.DataFrame(levene50pvals, levene100pvals, levene200pvals)

In [24]:
min(levene50pvals)

0.16296367712440338

In [25]:
min(levene100pvals)

0.026824692560992926

In [26]:
min(levene200pvals)

0.0012121081375865794

## Conclusion:  Based on levene's test we can conclude that there is a significant surge in volatility after the 200 day moving average is breached